In [621]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections
from collections import Counter
from itertools import combinations

In [622]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1568,tt0368891,100000000,347451894,National Treasure,Nicolas Cage|Diane Kruger|Justin Bartha|Sean B...,Jon Turteltaub,The greatest adventure history has ever revealed.,"Modern treasure hunters, led by archaeologist ...",131,Adventure|Action|Thriller|Mystery,Walt Disney Pictures|Jerry Bruckheimer Films|S...,11/19/2004,6.4,2004
751,tt1324999,110000000,712171856,The Twilight Saga: Breaking Dawn - Part 1,Kristen Stewart|Robert Pattinson|Taylor Lautne...,Bill Condon,Forever is just the beginning,The new found married bliss of Bella Swan and ...,117,Adventure|Fantasy|Romance,Summit Entertainment|Sunswept Entertainment|To...,3/15/2011,5.8,2011
642,tt0494238,60000000,57490374,Inkheart,Brendan Fraser|Sienna Guillory|Andy Serkis|Eli...,Iain Softley,Every story ever written is just waiting to be...,The adventures of a father and his young daugh...,106,Adventure|Family|Fantasy,New Line Cinema|Internationale Filmproduktion ...,12/11/2008,5.8,2008
1629,tt0381707,37000000,113086475,White Chicks,Shawn Wayans|Marlon Wayans|Jaime King|Frankie ...,Keenen Ivory Wayans,They're going deep undercover.,"Two FBI agent brothers, Marcus and Kevin Copel...",109,Comedy,Columbia Pictures|Revolution Studios|Wayans Br...,6/23/2004,6.4,2004
1080,tt2215719,12000000,32726956,Katy Perry: Part of Me,Katy Perry|Russell Brand|Shannon Woodward|Rach...,Dan Cutforth|Jane Lipsitz,Be yourself and you can be anything,Giving fans unprecedented access to the real l...,93,Documentary|Music,Paramount Pictures|Imagine Entertainment|MTV F...,6/28/2012,6.5,2012


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [623]:
answers = {} # создадим словарь для ответов

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides(tt1298650)' #+
answers['2'] = 'Gods and Generals(tt0279111)' #+
answers['3'] = 'Winnie the Pooh(tt1449283)' #+
answers['4'] = 110 #+
answers['5'] = 107 #+
answers['6'] = 'Avatar(tt0499549)' #+
answers['7'] = 'The Lone Ranger(tt1210819)' #+
answers['8'] = 1478 #+
answers['9'] = 'The Dark Knight' #+
answers['10'] = 'The Lone Ranger' #+
answers['11'] = 'Drama' #+
answers['12'] = 'Drama' #+
answers['13'] = 'Peter Jackson' #+
answers['14'] = 'Robert Rodriguez' #+
answers['15'] = 'Chris Hemsworth' #+
answers['16'] = 'Matt Damon' #+
answers['17'] = 'Action' #+
answers['18'] = 'K-19: The Widowmaker' #+
answers['19'] = 2015 #+
answers['20'] = 2014 #+
answers['21'] = 9 #+
answers['22'] = 450 #+
answers['23'] = 'Peter Jackson' #+
answers['24'] = 'Four By Two Productions' #+
answers['25'] = 'Midnight Picture Show' #+
answers['26'] = 'The Dark Knight', 'Inside Out', '12 Years a Slave'  #+
answers['27'] = 'Daniel Radcliffe', 'Rupert Grint' #+


# другие предобработки колонок:


data = pd.read_csv('movie_bd_v5.csv')
data_cast = pd.read_csv('movie_bd_v5.csv')
data_director = pd.read_csv('movie_bd_v5.csv')
data_genres = pd.read_csv('movie_bd_v5.csv')
data_pivot = pd.read_csv('movie_bd_v5.csv')
data_rel_date = pd.read_csv('movie_bd_v5.csv')
data_len_title = pd.read_csv('movie_bd_v5.csv')
data_len_word = pd.read_csv('movie_bd_v5.csv')
double_cast = pd.read_csv('movie_bd_v5.csv')


# прибыль
data['profit'] = data.revenue - data.budget

# разделение актеров
data_cast['cast'] = data_cast['cast'].apply(lambda x:x.split('|'))
data_cast_explode = data_cast.explode('cast')

# разделение режиссеров
data_director['director'] = data_director['director'].apply(lambda x:\
                                                            x.split('|'))
data_director = data_director.explode('director')

# разделение жанров
data_genres['genres'] = data_genres['genres'].apply(lambda x:x.split('|'))
data_genres = data_genres.explode('genres')

# разделение даты релиза
data_rel_date['release_date'] = data_rel_date['release_date'
                                             ].apply(lambda x: x.split('/'))
data_rel_date['month'] = data_rel_date['release_date'].apply(lambda x: x[0])

# разделение компаний
data_len_title['len_title'] = data_len_title.original_title.apply(lambda x:\
                                                                  len(x))
data_len_title['production_companies'] = data_len_title['production_companies'
                                                       ].apply(lambda x:\
                                                               x.split('|'))
explode_companies = data_len_title.explode('production_companies')

# разделение компаний по количеству слов в названиях фильмов
data_len_word['production_companies'
             ] = data_len_word['production_companies'].apply(lambda x:\
                                                             x.split('|'))
data_len_word['len_overview'] = data_len_word['overview'].apply(lambda x:\
                                                                len(x))
companies_word = data_len_word.explode('production_companies')

#создаем новый столбец с комбинациями актеров через itertools
double_cast['cast'] = double_cast['cast'].apply(lambda x:x.split('|'))
double_cast['new_cast'] = double_cast['cast'
                                     ].apply(lambda x:\
                                             list(combinations(sorted(x),2)))
double_cast_explode = double_cast.explode('new_cast')


#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [221]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [570]:
#вариант_1:
data_title = data[data.budget == 
                  data.budget.max()].describe(include = ['object'])\
                                             ['original_title']['top']
data_imdb_id = data[data.budget == 
                    data.budget.max()].describe(include = ['object'])\
                                               ['imdb_id']['top']
print(data_title, '(', data_imdb_id, ')', sep='')

Pirates of the Caribbean: On Stranger Tides(tt1298650)


In [571]:
#вариант_2:
data[data.budget == data.budget.max()][['original_title','imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [572]:
#вариант_3:

data.original_title[data.budget.idxmax()] 
data.imdb_id[data.budget.idxmax()] 

# объединяем название и id фильма:
print(data.original_title[data.budget.idxmax()],'(',\
      data.imdb_id[data.budget.idxmax()],')', sep='')

Pirates of the Caribbean: On Stranger Tides(tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [573]:
#вариант_1:

data_title = data[data.runtime == 
                  data.runtime.max()].describe(include = ['object'])\
                                              ['original_title']['top']
data_imdb_id = data[data.runtime == 
                    data.runtime.max()].describe(include = ['object'])\
                                                ['imdb_id']['top']

print(data_title, '(', data_imdb_id, ')', sep='')

Gods and Generals(tt0279111)


In [574]:
#вариант_2:
data[data.runtime == data.runtime.max()][['original_title','imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


In [575]:
#вариант_3:

data.original_title[data.runtime.idxmax()]
data.imdb_id[data.runtime.idxmax()]

# объединяем название и id фильма:
print(data.original_title[data.runtime.idxmax()],'(',\
      data.imdb_id[data.runtime.idxmax()],')', sep='')

Gods and Generals(tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [576]:
#вариант_1:

data_title = data[data.runtime == 
                  data.runtime.min()].describe(include = ['object'])\
                                              ['original_title']['top']
data_imdb_id = data[data.runtime == 
                    data.runtime.min()].describe(include = ['object'])\
                                                ['imdb_id']['top']

print(data_title, '(', data_imdb_id, ')', sep='')

Winnie the Pooh(tt1449283)


In [577]:
#вариант_2:
data[data.runtime == data.runtime.min()][['original_title','imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


In [578]:
#вариант_3:

data.original_title[data.runtime.idxmin()]
data.imdb_id[data.runtime.idxmin()]

# объединяем:
print(data.original_title[data.runtime.idxmin()],'(',\
      data.imdb_id[data.runtime.idxmin()],')', sep='')

Winnie the Pooh(tt1449283)


# 4. Какова средняя длительность фильмов?


In [579]:
int(data.describe()['runtime']['mean'].round()) #округляем и отбрасываем дробь

110

# 5. Каково медианное значение длительности фильмов? 

In [580]:
int(data.describe()['runtime']['50%'])

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [581]:
#profit получен в предобработке, что в начале
#вариант_1:
print((data[data.profit == 
                  data.profit.max()].describe(include = ['object'])\
                                              ['original_title']['top']),\
     '(',
     (data[data.profit == 
                  data.profit.max()].describe(include = ['object'])\
                                              ['imdb_id']['top']), \
     ')', sep='')

Avatar(tt0499549)


In [582]:
#вариант_2:
data = pd.read_csv('movie_bd_v5.csv')
data['profit'] = data.revenue - data.budget
data[data.profit == data.profit.max()][['original_title','imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


In [583]:
#profit получен в предобработке
#вариант_3:
data.original_title[data.profit.idxmax()]
data.imdb_id[data.profit.idxmax()]

# объединяем:
print(data.original_title[data.profit.idxmax()],'(',\
      data.imdb_id[data.profit.idxmax()],')', sep='')

Avatar(tt0499549)


# 7. Какой фильм самый убыточный? 

In [584]:
#profit получен в предобработке
#вариант_1:
print((data[data.profit == 
                  data.profit.min()].describe(include = ['object'])\
                                              ['original_title']['top']),\
     '(',
     (data[data.profit == 
                  data.profit.min()].describe(include = ['object'])\
                                              ['imdb_id']['top']), \
     ')', sep='')

The Lone Ranger(tt1210819)


In [585]:
#вариант_2:
data = pd.read_csv('movie_bd_v5.csv')
data['profit'] = data.revenue - data.budget
data[data.profit == data.profit.min()][['original_title','imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [586]:
#profit получен в предобработке
#вариант_3:
data.original_title[data.profit.idxmin()]
data.imdb_id[data.profit.idxmin()]

# объединяем:
print(data.original_title[data.profit.idxmin()],'(',\
      data.imdb_id[data.profit.idxmin()],')', sep='')

The Lone Ranger(tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [587]:
data[(data.revenue > data.budget)].imdb_id.nunique()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [588]:
#проверяем максимальный сбор (revenue) в 2008

data[(data.release_year == 2008)].revenue.max()

1001921825

In [589]:
'''вариант-1:ищем название фильма с максимальным сбором в 2008 году 
через условие'''

data[(data.revenue == 
     (data[(data.release_year == 2008)].revenue.max()))].original_title

599    The Dark Knight
Name: original_title, dtype: object

In [590]:
#вариант-2: вывод первой строки через head() с нужными столбцами

data[(data.release_year == 2008) & 
     (data.revenue.max())][['revenue','imdb_id','original_title']].head(1)

,revenue,imdb_id,original_title
599,1001921825,tt0468569,The Dark Knight


In [591]:
#вариант-3: вывод всей строки
data[data['revenue'] == data[(data.release_year==2008)].revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [592]:
#проверяем минимальный сбор за 2012-2014 годы

data[(data.release_year > 2011) & (data.release_year < 2015)].profit.min()

-165710090

In [593]:
#вариант-1:ищем название убыточного фильма за 2012-2014 

data[(data.profit == 
     (data[(data.release_year > 2011) & 
           (data.release_year < 2015)].profit.min()))].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

In [594]:
#вариант-2: вывод первой строки через head() с нужными столбцами

data[(data.release_year > 2011) & 
     (data.release_year < 2015) & 
     (data.profit == 
      data.profit.min())][['profit','imdb_id','original_title']].head(1)

,profit,imdb_id,original_title
1245,-165710090,tt1210819,The Lone Ranger


In [595]:
#вариант-3: вывод первой строки с тремя столбцами через head() и сортировку

data.loc[(data.release_year > 2011) & 
         (data.release_year < 2015)].sort_values(by='profit')\
         [['profit','imdb_id','original_title']].head(1)

,profit,imdb_id,original_title
1245,-165710090,tt1210819,The Lone Ranger


In [596]:
#вариант-4: вывод всей строки

data[data.profit == 
     data[(data.release_year > 2011) & 
          (data.release_year < 2015)].profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090


# 11. Какого жанра фильмов больше всего?

In [597]:
#разделение жанров и датасет data_genres получены в предобработке

Counter(data_genres.explode('genres')['genres']).most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [598]:
#разделяем жанры (получено в предобработке)
#находим прибыль(profit)
#в новом датасете ищем прибыль больше нуля
#группируем, считаем кол-во и находим первый(наибольший) элемент

data_genres['profit'] = data_genres['revenue'] - data_genres['budget']
data_profit = data_genres[data_genres.profit > 0]
group_genres = data_profit['genres'].value_counts() 
group_genres.index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [599]:
#Вариант_1:
data.groupby('director')['revenue'].sum().sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

In [600]:
#Вариант_2:

data.groupby('director')['revenue'].sum().sort_values(ascending=False).index[0]

'Peter Jackson'

In [601]:
#Вариант_3: с учетом разделения режиссеров (получено в предобработке)

data_director.groupby('director')['revenue'].sum().sort_values(ascending=False
                                                              ).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [602]:
#разделение режиссеров в датасете data_director получено в предобработке

data_action = data_director[data_director.genres.str.contains('Action')]
data_action.director.value_counts().index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [603]:
#разделение актеров в датасете data_cast_explode получено в предобработке
#фильтруем по году рилиза = 2012
#сумируем по сборам, сортируем по убыванию и находим первый элемент

data_cast_explode = data_cast_explode[(data_cast_explode.release_year == 2012)]

group = data_cast_explode.groupby(['cast'])['revenue'].sum()
group.sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [604]:
data = pd.read_csv('movie_bd_v5.csv')
data = data[(data.budget > data.budget.mean())]

data['cast'] = data['cast'].apply(lambda x:x.split('|'))
data_explode_cast = data.explode('cast')

group = data_explode_cast.groupby(['cast'])['imdb_id'].count()
group.sort_values(ascending=False).index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [605]:
data_Cage = pd.read_csv('movie_bd_v5.csv')

data_Cage = data_Cage[data_Cage.cast.str.contains('Nicolas Cage')]

data_Cage['genres'] = data_Cage['genres'].apply(lambda x:x.split('|'))
Cage_genres = data_Cage.explode('genres')

group = Cage_genres.groupby(['genres'])['imdb_id'].count()
group.sort_values(ascending=False).index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [606]:
#вариант_1: вывод всей строки

data['profit'] = data.revenue - data.budget

data = data[data.production_companies.str.contains('Paramount Pictures')]
data[(data['profit'] == data.profit.min())]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
925,tt0267626,100000000,35168966,K-19: The Widowmaker,"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034


In [607]:
#вариант_2: вывод названия

data = data[data.production_companies.str.contains('Paramount Pictures')]
data[(data['profit'] == data.profit.min())]['original_title'].iloc[0]

'K-19: The Widowmaker'

In [608]:
#вариант_3: вывод нескольких столбцов

data = data[data.production_companies.str.contains('Paramount Pictures')]
data[(data['profit'] == 
      data.profit.min())][['original_title','imdb_id','profit']]

,original_title,imdb_id,profit
925,K-19: The Widowmaker,tt0267626,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [609]:
#data_pivot отпределено в предобработке

pivot = data_pivot.pivot_table(values='revenue',
                               columns='release_year',
                               aggfunc='sum',
                               fill_value=0)

display(pivot.max().sort_values(ascending=False).index[0])

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [610]:
data_pivot = data_pivot[data_pivot.production_companies.str.contains('Warner Bros')]

pivot = data_pivot.pivot_table(values='revenue',
                               columns='release_year',
                               aggfunc='sum',
                               fill_value=0)

display(pivot.max().sort_values(ascending=False).index[0])

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [611]:
#разделение даты релиза data_rel_date определено в предобработке

pivot = data_rel_date.pivot_table(values='imdb_id',
                                  columns='month',
                                  aggfunc='count',
                                  fill_value=0)
display(pivot.max().sort_values(ascending=False).index[0])

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [612]:
#разделение даты релиза data_rel_date определено в предобработке

summer = data_rel_date[data_rel_date.month.str.contains('6|7|8')]

summer.month.describe().iloc[0]
#или
summer.month.describe().loc['count']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [613]:
#разделение даты релиза data_rel_date определено в предобработке
#вариант_1:

winter = data_rel_date[~data_rel_date.month.str.contains('3|4|5|6|7|8|9|10|11')]

winter.director.describe().loc['top']

'Peter Jackson'

In [614]:
#разделение даты релиза data_rel_date определено в предобработке
#вариант_2: через collections

winter = data_rel_date[~data_rel_date.month.str.contains('3|4|5|6|7|8|9|10|11')]

collections.Counter(winter['director'].str.cat(sep ='|'
                                              ).split('|')).most_common(1)

[('Peter Jackson', 7)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [615]:
#создаем новую колонку с длиной символов, применив apply с функцией len
#новая колонка data['len_title'] определена в предобработке
#разделяем компании (определено в предобработке)
#находим среднее кол-во длины символов в названии фильмов компаний
#сортируем по убыванию и находим первый(наибольший) элемент

group = explode_companies.groupby('production_companies')['len_title'].mean()
group.sort_values(ascending=False).index[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [616]:
'''разделение компаний по количеству слов в названиях фильмов (companies_word)
определено в предобработке'''

companies_word.groupby('production_companies'
                         ).len_overview.mean().sort_values(ascending=False
                                                          ).index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [617]:
data_best = pd.read_csv('movie_bd_v5.csv')
best = data_best.vote_average.quantile(0.99) #нашли 1% через квантиль
df = data_best[data_best.vote_average > best][['original_title','vote_average']]
df.sort_values(by='vote_average', ascending=False)

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
119,Guardians of the Galaxy,7.9
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [618]:
'''создаем новый столбец 'new_cast' с комбинациями актеров через itertools,
получаем новый разделенный датасет double_cast_explode, определено 
в предобработке.
'''

group = double_cast_explode.groupby(['new_cast'])['imdb_id'].count()

#вывод результата:
display(group.sort_values(ascending=False).idxmax())
#или
display(group.sort_values(ascending=False).head())

('Daniel Radcliffe', 'Rupert Grint')

new_cast
(Daniel Radcliffe, Rupert Grint)       8
(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Emma Watson)        8
(Helena Bonham Carter, Johnny Depp)    6
(Ben Stiller, Owen Wilson)             6
Name: imdb_id, dtype: int64

# Submission

In [619]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides(tt1298650)',
 '2': 'Gods and Generals(tt0279111)',
 '3': 'Winnie the Pooh(tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar(tt0499549)',
 '7': 'The Lone Ranger(tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ('The Dark Knight', 'Inside Out', '12 Years a Slave'),
 '27': ('Daniel Radcliffe', 'Rupert Grint')}

In [620]:
# и убедиться что ни чего не пропустил)
len(answers)

27